In [13]:
###########
# PRELUDE #
###########

# auto-reload changed python files
%load_ext autoreload
%autoreload 2

# Format cells with %%black
%load_ext blackcellmagic

# nice interactive plots
%matplotlib inline

import matplotlib
# enable more math expressions in matplotlib labels
matplotlib.rcParams['text.latex.preamble'] = r"\usepackage{amsmath}"
# no blurry plots!
%config InlineBackend.figure_format = 'retina'

# add repository directory to include path
from pathlib import Path
import sys
PROJECT_DIR = Path('../..').resolve()
sys.path.append(str(PROJECT_DIR))

import inspect
def _acceptable_global(name, value):
    """Returns True if a global variable with name/value can be safely ignored"""
    return (
        # stuff that's normal to share everywhere
        inspect.isroutine(value) or
        inspect.isclass(value) or
        inspect.ismodule(value) or
        # leading underscore marks private variables
        name.startswith('_') or
        # all-caps names indicate constants
        name.upper() == name or
        # ignore IPython stuff
        name in {'In', 'Out'} or 
        getattr(value, '__module__', '').startswith('IPython'))

def assert_globals_clean():
    """Raises an assertion error if there are unmanaged global variables.
       Variables that are considered 'managed' include those formatted with 
       ALL_CAPS (constants), _a_leading_underscore (recognized as a global but at
       least indicated as private to the cell), classes and modules, automatic
       imports from IPython, and functions generally."""
    unmanaged_globals = {k:type(v) for k, v in globals().items() if not _acceptable_global(k, v)}
    if unmanaged_globals != {}:
        raise AssertionError(f"Unmanaged globals found: {unmanaged_globals}")
    ok("No unmanaged globals detected")

from IPython.display import display, Markdown, HTML

def markdown(s):
    return display(Markdown(s))

def html(s):
    return display(HTML(s))

def ok(message="OK"):
    html(f"<div class=\"alert alert-block alert-success\">{message}</div>")

html("""
<style>
.custom-assignment-text {
    background-color: lightyellow;
    border: 1px solid darkkhaki; 
    padding: 10px;
    border-radius: 2px
}
</style>""")

# Fixes space left behind tqdm progress bars with leave=False
# see https://github.com/jupyterlab/jupyterlab/issues/7354
html("""
<style>
.jp-OutputArea-prompt:empty {
  padding: 0;
  border: 0;
}
</style>
""")

def display_table(data, title, headers):
    """Display data in an HTML table inline in the notebook
       data: list of lists of values to put in table rows
       title: to set table caption
       headers: list of table header strings"""
    text = "<table>"
    text += f"<caption style='font-weight: bold; font-size: large'>{title}</caption>"
    
    text += "<tr>"
    for h in headers:
        text += f'<th style="text-align:center">{h}</th>'
    text += "</tr>"
    
    for row in data:
        text += "<tr>"
        for value in row:
            text += f"<td>{value}</td>"
        text += "</tr>"
    text += "</table>"
    html(text)

markdown("#### Custom functionality enabled:")
markdown("* Format a code cell by entering `%%black` at the top of it")
markdown("* Surround markdown cells with  `<div class=\"custom-assignment-text\">\\n\\n ... \\n\\n</div>` to format course-provided assignment text")
markdown("* Use `ok(<message>)` to notify of a passing test")
markdown("* Use `assert_globals_clean()` to check that all globals are managed (private, constants, etc.)")
markdown("* Use `display_table` to display data in an inline HTML table")

#### Custom functionality enabled:

* Format a code cell by entering `%%black` at the top of it

* Surround markdown cells with  `<div class="custom-assignment-text">\n\n ... \n\n</div>` to format course-provided assignment text

* Use `ok(<message>)` to notify of a passing test

* Use `assert_globals_clean()` to check that all globals are managed (private, constants, etc.)

* Use `display_table` to display data in an inline HTML table

<div class="custom-assignment-text">

# Part 1: Spectral Methods Intuition

## Goal

In this exercise you will build some intuition for the eigenvectors of various simple graphs.

</div>

In [4]:
import numpy

<div class="custom-assignment-text">
    
## Description

(a) (6 points) Consider the graphs as given in Figure 1. For this part, take $n = 6$. For each graph write down the Laplacian matrix $L = D −A$ where $D$ is the diagonal matrix with entry $D_{i,j}$ being the degree of the $i$th node, and $A$ is the adjacency matrix, with entry $A_{i,j} = 1$ if there is an edge between nodes $i$ and $j$, and $A_{i,j} = 0$ otherwise. Your answer should be in the form of actual matrices (i.e., not just English descriptions of matrices).
    
<figure><center><img src="../../materials/Week 6/Figure_1.png" width=100%><figcaption>Figure 1: The graphs for question 1a.</figcaption></center></figure>

</div>

In [8]:
class LaplacianGraph():
    def __init__(self, n):
        self.n = n
        self.matrix = numpy.zeros((n, n))
    def add_edge(self, i, j, weight=1):
        self.matrix[i,i] += weight
        self.matrix[j,j] += weight
        self.matrix[i,j] -= weight
        self.matrix[j,i] -= weight

In [32]:
def line_graph(n=6):
    graph = LaplacianGraph(n)
    for i in range(n-1):
        graph.add_edge(i, i+1)
    return graph

def line_with_point_graph(n=6):
    graph = line_graph(n)
    for i in range(n-2):
        graph.add_edge(i, n-1)
    return graph

def circle_graph(n=6):
    graph = line_graph(n)
    # close the circle
    graph.add_edge(0, n-1)
    return graph

def circle_with_point_graph(n=6):
    graph = line_graph(n)
    # close the circle
    graph.add_edge(0, n-2)
    # connect points to center (n-2 is already connected to n-1)
    for i in range(0, n-2):
        graph.add_edge(i, n-1)
    return graph
    

def test_graphs(graph_constructors):
    for name, f in graph_constructors.items():
        n = 100
        matrix = f(n).matrix
        assert matrix.shape == (n,n), f'{name}: {matrix.shape}'
        # graph should be symmetric
        assert numpy.allclose(matrix, matrix.T)
        # diagonal should be non-negative, others non-positive
        for i in range(n):
            for j in range(n):
                if i == j:
                    assert matrix[i,j] >= 0, f'{name}: {matrix[i,j]}'
                else:
                    assert matrix[i,j] <= 0, f'{name}: {matrix[i,j]}'
        
        
test_graphs({"Line graph": line_graph, "line with point": line_with_point_graph, "circle": circle_graph, "circle with point": circle_with_point_graph})
ok()

In [33]:
markdown("* (a) Line graph")
print(line_graph().matrix)
markdown("* (b) Line graph with added point")
print(line_with_point_graph().matrix)
markdown("* (c) Circle graph")
print(circle_graph().matrix)
markdown("* (d) Circle graph with added point")
print(circle_with_point_graph().matrix)

* (a) Line graph

[[ 1. -1.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.]
 [ 0.  0.  0. -1.  2. -1.]
 [ 0.  0.  0.  0. -1.  1.]]


* (b) Line graph with added point

[[ 2. -1.  0.  0.  0. -1.]
 [-1.  3. -1.  0.  0. -1.]
 [ 0. -1.  3. -1.  0. -1.]
 [ 0.  0. -1.  3. -1. -1.]
 [ 0.  0.  0. -1.  2. -1.]
 [-1. -1. -1. -1. -1.  5.]]


* (c) Circle graph

[[ 2. -1.  0.  0.  0. -1.]
 [-1.  2. -1.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.]
 [ 0.  0.  0. -1.  2. -1.]
 [-1.  0.  0.  0. -1.  2.]]


* (d) Circle graph with added point

[[ 3. -1.  0.  0. -1. -1.]
 [-1.  3. -1.  0.  0. -1.]
 [ 0. -1.  3. -1.  0. -1.]
 [ 0.  0. -1.  3. -1. -1.]
 [-1.  0.  0. -1.  3. -1.]
 [-1. -1. -1. -1. -1.  5.]]
